In [ ]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install nltk
# !pip install gspread
# !pip install gspread-pandas
# !pip install apache-airflow
# !pip install dvc

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

def Data_Extraction_dawn():
    base_url = 'https://www.dawn.com/latest-news'
    urls = []

    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract article links
    article_links = soup.findAll('a', {'class':'story__link'})

    for link in article_links:
        urls.append(link['href'])
    
    print(urls)
    # Open CSV file
    with open('news.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Title", "Body", "Source"])
        id=0
    # Loop through each URL
        for url in urls:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find('a', {'class': 'story__link'})
            p_tags = soup.find('div', {'class': 'story__content overflow-hidden text-4 sm:text-4.5 pt-1 mt-1'})
            if p_tags is None or title is None:
                continue
            else:
                title=title.getText()
                p_tags=p_tags.find_all('p')
            body=' '.join([p.get_text(strip=True) for p in p_tags])
            print(title)
            # write to csv
            id=id+1
            writer.writerow([id, title, body, url])

    print("Scraping completed and data stored in news.csv")

def Data_Extraction_bbc():
    parent = "https://www.bbc.com/"
    base_url = 'https://www.bbc.com/news/world/europe'
    urls = []

    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract article links
    article_links = soup.find('div', {'class':'sc-5b94fa74-0 sc-da05643e-1 dzHQkZ djXsFQ'})
    print(article_links)
    arti=article_links.findAll("a")
    for a in arti:
        print(a["href"])

# Open CSV file
    with open('news.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        id=0
        # Loop through each URL
        for url in arti:
            reqUrl=parent+url["href"]
            print(reqUrl)
            response = requests.get(reqUrl)
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find('div', {'class':"sc-b77c8bb-0 kqIEbK"})
            p_tags = soup.find('div', {'class': 'sc-b77c8bb-0 daiYje'})
            if p_tags is None or title is None:
                continue
            else:
                title=title.getText()
                p_tags=p_tags.find_all('p')
                body=' '.join([p.get_text(strip=True) for p in p_tags])
                print(title)
        # write to csv
            id=id+1
            writer.writerow([id, title, body, url])
    print("Scraping completed and data stored in news.csv")
Data_Extraction_dawn()
Data_Extraction_bbc()

['https://www.dawn.com/news/1833056/situation-in-ajk-calms-down-after-2-days-of-violent-clashes', 'https://www.dawn.com/news/1833053/imf-mission-to-meet-pakistani-authorities-next-week-to-discuss-next-phase-of-engagement-official', 'https://www.dawn.com/news/1833052/fight-till-the-last-ball-pcb-chief-tells-cricket-team-in-dublin', 'https://www.dawn.com/news/1833050/no-such-thing-as-strategic-state-owned-enterprises-says-finance-minister-aurangzeb', 'https://www.dawn.com/news/1332592', 'https://www.dawn.com/news/1833046/second-night-of-auroras-seen-amid-extreme-solar-storm', 'https://www.dawn.com/news/1833042/canada-arrests-fourth-indian-national-in-killing-of-sikh-activist', 'https://www.dawn.com/news/1833021', 'https://www.dawn.com/news/1833014', 'https://www.dawn.com/news/1832994', 'https://www.dawn.com/news/1832981', 'https://www.dawn.com/news/1832982', 'https://www.dawn.com/news/1832974', 'https://www.dawn.com/news/1832987', 'https://aurora.dawn.com/news/1145113', 'https://www.dawn

In [4]:
import pandas as pd
def Transform_data():
    # load .csv file into a DataFrame
    df = pd.read_csv('news.csv')
    # remove duplicates
    df = df.drop_duplicates()
    # drop nulls (this will drop any row with at least a single null value)
    df = df.dropna()

In [5]:

import gspread
from dvc.repo import Repo

def Store_data(cleaned_data):
    gc = gspread.service_account(filename='news.csv')
    sh = gc.create('News Data')
    worksheet = sh.get_worksheet(0)

    for data in cleaned_data:
        worksheet.append_row(list(data.values()))

    Repo.init()
    Repo.add('News Data')
    Repo.commit("-m Data uploaded")
    Repo.push("origin")

In [ ]:

from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

default_args = {
    'start_date': datetime(2022, 1, 1),
}

def extract_data():
    Data_Extraction_dawn()
    Data_Extraction_bbc()
    pass

def transform_data():
    Transform_data()
    pass

def store_data():
    Store_data()
    pass

with DAG('big_data_project', default_args=default_args, schedule_interval='@daily') as dag:
    extract_data = PythonOperator(
        task_id='extract_data',
        python_callable=extract_data
    )

    transform_data = PythonOperator(
        task_id='transform_data',
        python_callable=transform_data
    )

    store_data = PythonOperator(
        task_id='store_data',
        python_callable=store_data
    )

    # define task dependencies
    extract_data >> transform_data >> store_data